In [2]:
let ResolutionFolder = __SOURCE_DIRECTORY__

[<RequireQualifiedAccess>]
type Log =
| ChangeDir of string
| GoUp
| GoRoot
| File of int*string
| Dir of string

type AFile = {
    name: string
    size: int
}
type ADir = {
    name: string
    mutable parent: ADir option
    mutable dirs: Set<ADir>
    mutable files: Set<AFile>
} with
    static member New (name, scope) = { name = name; parent = scope; dirs = Set.empty; files = Set.empty }
    member this.Size() =
        let files = (this.files |> Seq.sumBy( fun it -> it.size ))
        let dirs = (this.dirs |> Seq.sumBy (fun it -> it.Size()))
        files + dirs
    member this.AddFile(size, name) =
        this.files <- this.files |> Set.add { size = size; name = name }
        Some this
    member this.AddDir(name) =
        this.dirs <- this.dirs |> Set.add (ADir.New(name, (Some this)))
        Some this
    member this.ChangeDir(name) =
        this.dirs |> Seq.tryFind(fun it -> it.name = name )
    member this.GoUp() =
        this.parent |> Option.orElse(Some this)
    member this.GoToRoot() =
        if this.parent.IsNone then 
            Some this
        else 
            this.parent |> Option.bind( fun it -> it.GoToRoot())

let parser (state: ADir option) (line: Log) =
    match state with 
    | None -> None
    | Some folder -> 

    match line with
    | Log.GoRoot ->
        folder.GoToRoot()
    | Log.ChangeDir name -> 
        folder.ChangeDir name
    | Log.GoUp -> 
        folder.GoUp()
    | Log.File (size,name) -> 
        folder.AddFile(size, name)
    | Log.Dir name -> 
        folder.AddDir name

let parseLog (line:string) =
    match line.Split(" ") with
    | [| "$"; "cd"; "/" |] -> Log.GoRoot |> Some
    | [| "$"; "cd"; ".." |] -> Log.GoUp |> Some
    | [| "$"; "cd"; dir |] -> Log.ChangeDir dir |> Some
    | [| "dir"; dir |] -> Log.Dir dir |> Some
    | [| size; file |] when (Int64.TryParse(size)) |> fst -> Log.File (size |> int ,file) |> Some
    | unknown ->  None

let rec find (fs:ADir) =
    seq{
        yield fs.Size()
        let children = 
            fs.dirs
            |> Seq.map find
            |> Seq.concat
        yield! children
    }

let maybeRoot =
    File.ReadLines(ResolutionFolder + "/input7.txt")
    |> Seq.map parseLog
    |> Seq.choose id
    |> Seq.fold parser (ADir.New("/", None) |> Some)
    |> Option.bind (fun it -> it.GoToRoot())

let part1 =
    match maybeRoot with
    | None -> 0
    | Some root ->
        root |> find |> Seq.filter(fun it -> it <= 100000) |> Seq.sum

let part2 =
    match maybeRoot with
    | None -> 0
    | Some root ->
        let left = 70000000 - root.Size()
        let target = 30000000 - left
        root |> find |> Seq.filter(fun it -> it >= target) |> Seq.min

part2

index,value
0,1400951
1,101940
2,101940
3,752508
4,1978542
5,30443
6,30443
7,918045
8,272548
9,165670
